In [1]:
# default_exp pictures

# Pictures

>  Create List with all pictures

Will use a csv file to store the list data

In [2]:
#hide
from nbdev.showdoc import *

In [3]:
#export 

import os
import pandas as pd



Here is what the documentation + code looks like for Card:

In [4]:
directory = r"/media/mycloud/My Pictures"

for (dirpath, dirnames, filenames) in os.walk(directory):
    print(dirpath)
    print(dirnames)
    print(filenames)
    break

/media/mycloud/My Pictures
['Pics_20120130', 'My Scans', 'IPhone_Mama_FOTOS', '27.09.04', 'TMP', 'Oma und Opa', 'Album Art', 'MCC Opening Mexico', 'iCloud Photos', 'Hochzeit', 'Galaxy Note II', 'Family', 'Casamento Andreia', 'BlackBerry', 'Arthur', 'Andrea Haus', 'Ana', 'Aline', '2006_11_25', '2006_11_20', '2006_11_09', 'moved_from_root', 'Saved Pictures', 'ControlCenter4', 'Camera Roll', 'Angelina']
['WirthFamily.JPG', 'TralhandonaSAP.JPG', 'Tchu.JPG', 'Tchu&Rosinha.JPG', 'Tchu e eu.JPG', 'Rosinha.JPG', 'portugueiz.jpg', 'foto.JPG', 'Procuração Ana Cláudia.doc', 'photo.JPG', 'Pernilzinho.JPG', 'Pernil.JPG', 'Passport_foto-p1.jpg', 'Passbild_Alois-p1.jpg', 'OlhinhosdeSono.JPG', 'OFotografo3.JPG', 'OFotografo2.JPG', 'OFotografo.JPG', 'meninas_aprontando.jpg', 'Malaga_sol.jpg', 'LunchnaSAP.JPG', 'LunchatSAP.JPG', 'MCC_SOS_App_Favicon.png', 'IMG_1898.jpg', 'IMG00347-20130214-0905.jpg', 'IMG_0830.JPG', 'Hannah&Judith.JPG', 'foto (2).JPG', 'Fot5A9759.jpg', 'Fot2A9759.jpg', 'Fot1A9759.jpg', 

In [6]:
data = {
    'pictures': filenames,
    'paths': dirpath
}

df = pd.DataFrame(data)
df

,pictures,paths
0,WirthFamily.JPG,/media/mycloud/My Pictures
1,TralhandonaSAP.JPG,/media/mycloud/My Pictures
2,Tchu.JPG,/media/mycloud/My Pictures
3,Tchu&Rosinha.JPG,/media/mycloud/My Pictures
4,Tchu e eu.JPG,/media/mycloud/My Pictures
...,...,...
336,IMG_2394.JPG,/media/mycloud/My Pictures
337,64AA2C01-419F-4164-84A1-C22D6F81350D-1568-0000...,/media/mycloud/My Pictures
338,IMG_0051.JPG,/media/mycloud/My Pictures
339,290f245a-b94d-49e4-b99a-92d0d03062f1.jpg,/media/mycloud/My Pictures


In [10]:
stat_data = []
entry_data = []
with os.scandir(directory) as dir_content:
    for entry in dir_content:
        info = entry.stat()        
        stat_data.append(info)
        entry_data.append(entry)

print(len(stat_data))
print(len(entry_data))

367
367


In [11]:
df1 = pd.DataFrame(entry_data)
df2 = pd.DataFrame(stat_data)
df = pd.concat([df1, df2], axis =1)
df

,0,0,1,2,3,4,5,6,7,8,9
0,<DirEntry 'Pics_20120130'>,16877,157988103,79,2,0,0,0,1547898654,1547898654,1547898654
1,<DirEntry 'My Scans'>,16877,158907182,79,2,0,0,0,1547899900,1547899900,1547899900
2,<DirEntry 'IPhone_Mama_FOTOS'>,16877,158907185,79,2,0,0,0,1547899519,1547899519,1547899519
3,<DirEntry '27.09.04'>,16877,158907189,79,2,0,0,0,1547899894,1547899894,1547899894
4,<DirEntry 'TMP'>,16877,158907194,79,2,0,0,0,1589826397,1589826397,1589826397
...,...,...,...,...,...,...,...,...,...,...,...
362,<DirEntry 'IMG_2394.JPG'>,33261,157631698,79,1,0,0,21907,1580295076,1580295076,1580295076
363,<DirEntry '64AA2C01-419F-4164-84A1-C22D6F81350...,33261,157631699,79,1,0,0,32829,1582990726,1582990726,1582990726
364,<DirEntry 'IMG_0051.JPG'>,33261,157631700,79,1,0,0,3432385,1533476216,1533476216,1533476216
365,<DirEntry '290f245a-b94d-49e4-b99a-92d0d03062f...,33261,157631701,79,1,0,0,99750,1553886249,1553886249,1553886249
